# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.utils.validation import check_is_fitted
# from keras.activations import leaky_relu
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.optimizers import Adam, SGD, RMSprop
# from tensorflow.keras.regularizers import l1_l2, l2, l1
# from tensorflow.keras.optimizers.schedules import ExponentialDecay
# from tensorflow.keras.layers import Input, BatchNormalization, Dense, Activation, Dropout, LeakyReLU, Conv1D, Concatenate, GlobalAveragePooling1D, LeakyReLU, Flatten

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/DATA 441

/content/drive/MyDrive/Colab Notebooks/DATA 441


In [4]:
from nadaraya_watson import NadarayaWatson, NadarayaWatsonCV

In [5]:
scaler = StandardScaler()

In [6]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [7]:
data = pd.read_csv('data/telecom_churn.csv')

In [8]:
X = data.loc[:, 'AccountWeeks':].values
y = data['Churn'].values

# Definitions

In [9]:
def get_natural_cubic_spline_model(x, y, minval=None, maxval=None, n_knots=None, knots=None):
    """
    Get a natural cubic spline model for the data.

    For the knots, give (a) `knots` (as an array) or (b) minval, maxval and n_knots.

    If the knots are not directly specified, the resulting knots are equally
    space within the *interior* of (max, min).  That is, the endpoints are
    *not* included as knots.

    Parameters
    ----------
    x: np.array of float
        The input data
    y: np.array of float
        The outpur data
    minval: float 
        Minimum of interval containing the knots.
    maxval: float 
        Maximum of the interval containing the knots.
    n_knots: positive integer 
        The number of knots to create.
    knots: array or list of floats 
        The knots.

    Returns
    --------
    model: a model object
        The returned model will have following method:
        - predict(x):
            x is a numpy array. This will return the predicted y-values.
    """

    if knots:
        spline = NaturalCubicSpline(knots=knots)
    else:
        spline = NaturalCubicSpline(max=maxval, min=minval, n_knots=n_knots)

    p = Pipeline([
        ('nat_cubic', spline),
        ('regression', LinearRegression(fit_intercept=True))
    ])

    p.fit(x, y)

    return p


class AbstractSpline(BaseEstimator, TransformerMixin):
    """Base class for all spline basis expansions."""

    def __init__(self, max=None, min=None, n_knots=None, n_params=None, knots=None):
        if knots is None:
            if not n_knots:
                n_knots = self._compute_n_knots(n_params)
            knots = np.linspace(min, max, num=(n_knots + 2))[1:-1]
            max, min = np.max(knots), np.min(knots)
        self.knots = np.asarray(knots)

    @property
    def n_knots(self):
        return len(self.knots)

    def fit(self, *args, **kwargs):
        return self


class NaturalCubicSpline(AbstractSpline):
    """Apply a natural cubic basis expansion to an array.
    The features created with this basis expansion can be used to fit a
    piecewise cubic function under the constraint that the fitted curve is
    linear *outside* the range of the knots..  The fitted curve is continuously
    differentiable to the second order at all of the knots.
    This transformer can be created in two ways:
      - By specifying the maximum, minimum, and number of knots.
      - By specifying the cutpoints directly.  

    If the knots are not directly specified, the resulting knots are equally
    space within the *interior* of (max, min).  That is, the endpoints are
    *not* included as knots.
    Parameters
    ----------
    min: float 
        Minimum of interval containing the knots.
    max: float 
        Maximum of the interval containing the knots.
    n_knots: positive integer 
        The number of knots to create.
    knots: array or list of floats 
        The knots.
    """

    def _compute_n_knots(self, n_params):
        return n_params

    @property
    def n_params(self):
        return self.n_knots - 1

    def transform(self, X, **transform_params):
        X_spl = self._transform_array(X)
        if isinstance(X, pd.Series):
            col_names = self._make_names(X)
            X_spl = pd.DataFrame(X_spl, columns=col_names, index=X.index)
        return X_spl

    def _make_names(self, X):
        first_name = "{}_spline_linear".format(X.name)
        rest_names = ["{}_spline_{}".format(X.name, idx)
                      for idx in range(self.n_knots - 2)]
        return [first_name] + rest_names

    def _transform_array(self, X, **transform_params):
        X = X.squeeze()
        try:
            X_spl = np.zeros((X.shape[0], self.n_knots - 1))
        except IndexError: # For arrays with only one element
            X_spl = np.zeros((1, self.n_knots - 1))
        X_spl[:, 0] = X.squeeze()

        def d(knot_idx, x):
            def ppart(t): return np.maximum(0, t)

            def cube(t): return t*t*t
            numerator = (cube(ppart(x - self.knots[knot_idx]))
                         - cube(ppart(x - self.knots[self.n_knots - 1])))
            denominator = self.knots[self.n_knots - 1] - self.knots[knot_idx]
            return numerator / denominator

        for i in range(0, self.n_knots - 2):
            X_spl[:, i+1] = (d(i, X) - d(self.n_knots - 2, X)).squeeze()
        return X_spl

In [ ]:
def boosted(x, y, xnew, model1, model2):
  model1.fit(x,y)
  residuals1 = y - model1.predict(x)
  model2.fit(x,residuals1)
  output = model1.predict(xnew) + model2.predict(xnew)
  return output 

# Model

## H function

In [ ]:

def H(  inputs, num_filters , dropout_rate ):
    x = BatchNormalization( epsilon=eps )( inputs )
    x = Activation('relu')(x)
    x = Conv1D(num_filters, kernel_size=(1), use_bias=False , kernel_initializer='he_normal' )(x)
    x = Dropout(rate=dropout_rate )(x)
    return x


## Transition Layer

In [ ]:
def transition(inputs, num_filters , compression_factor , dropout_rate ):
    # compression_factor is the 'θ'
    x = BatchNormalization( epsilon=eps )(inputs)
    x = Activation('relu')(x)
    num_feature_maps = inputs.shape[1] # The value of 'm'

    x = Conv1D( np.floor( compression_factor * num_feature_maps ).astype(int) ,
                               kernel_size=(1), use_bias=False, padding='same' , kernel_initializer='he_normal' , kernel_regularizer=l2( 1e-4 ) )(x)
    x = Dropout(rate=dropout_rate)(x)
    
    # x = AveragePooling2D(pool_size=(2, 2))(x)
    return x


## Dense Block

In [ ]:
def dense_block( inputs, num_layers, num_filters, growth_rate , dropout_rate ):
    for i in range(num_layers): # num_layers is the value of 'l'
        conv_outputs = H(inputs, num_filters , dropout_rate )
        inputs = Concatenate()([conv_outputs, inputs])
        num_filters += growth_rate # To increase the number of filters for each layer.
    return inputs, num_filters


# GAM

In [10]:
class GAM1:
    def __init__(self, n_knots=40):
        self.n_knots = 40
    
    def fit(self, x, y):
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new, y_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        n_knots = self.n_knots
        residuals = []
        for i in range(x.shape[1]):
          model = get_natural_cubic_spline_model(x[:,i], y, minval=np.min(x[:,i]), maxval=np.max(x[:,i]), n_knots=n_knots)
          yhat_test = model.predict(x_new[:,i])
          residuals.append(y_new-yhat_test)
        #return np.sum(residuals, axis=0)
        return np.mean(residuals, axis=0)

    def get_params(self, deep=True):
        return {"n_knots": self.n_knots}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [11]:
class GAM2:
    def __init__(self, n_knots=40):
        self.n_knots = 40
    
    def fit(self, x, y):
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new, y_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        n_knots = self.n_knots
        residuals = []
        for i in range(x.shape[1]):
          model = get_natural_cubic_spline_model(x[:,i], y, minval=np.min(x[:,i]), maxval=np.max(x[:,i]), n_knots=n_knots)
          yhat_test = model.predict(x_new[:,i])
          residuals.append(y_new-yhat_test)
        #return np.sum(residuals, axis=0)
        return np.mean(residuals, axis=0)

    def get_params(self, deep=True):
        return {"n_knots": self.n_knots}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

 # Best Models

In [ ]:
input_shape = ( 10, 1 ) 
num_blocks = 4
num_layers_per_block = 4
growth_rate = 2
dropout_rate = 0.2
compress_factor = 0.5
eps = 1.1e-5

num_filters = 16

inputs = Input( shape=input_shape )
outputs = Conv1D( num_filters , kernel_size=( 3 ) , use_bias=False, kernel_initializer='he_normal' , kernel_regularizer=l2( 1e-4 ) )( inputs )

for i in range( num_blocks ):
    outputs, num_filters = dense_block(outputs, num_layers_per_block , num_filters, growth_rate , dropout_rate )
    outputs = transition(outputs, num_filters , compress_factor , dropout_rate )

outputs = Flatten()(outputs)
# x = Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01))(x)
# maybe flatten and add a "thinking" layer like gelu
# x = GlobalAveragePooling2D()( x ) 

# x = Dense(2)(x)
# outputs = Activation('softmax')(x)

outputs = Dense( 2 )( outputs ) # sigmoid 1 softmax 2
outputs = Activation( 'softmax' )( outputs )

dn = Model( inputs , outputs )

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=1e-3/200)
optimizer = Adam(learning_rate=lr_schedule)

nn = Sequential()
# model.add(Dense(16,activation=LeakyReLU(alpha=0.08),use_bias=True,kernel_regularizer=l1_l2(l1=0.1,l2=0.2)))
# model.add(Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.01,l2=0.02)))
nn.add(BatchNormalization())
nn.add(Dense(32,activation=LeakyReLU(alpha=0.07),kernel_initializer='he_normal',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())
nn.add(Dense(64,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())
nn.add(Dense(32,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
nn.add(Dropout(0.2))
nn.add(BatchNormalization())
nn.add(Dense(16,activation='gelu',use_bias=True,kernel_regularizer=l1_l2(l1=0.001,l2=0.01)))
# nn.add(Dense(1, activation="sigmoid"))
nn.add(Dense(2, activation="softmax"))
callback = EarlyStopping(monitor='val_loss', patience=2)

forest1 = RandomForestClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=2, n_estimators=95)
forest2 = RandomForestRegressor(max_depth=6, min_samples_leaf=2, min_samples_split=2, n_estimators=65)
gam1 = GAM1(40)
gam2 = GAM2(40)
nw = NadarayaWatsonCV(dict(kernel=["polynomial"],gamma=np.logspace(-1, 1, 100)),scoring='neg_mean_absolute_error')
nn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[ 'accuracy' ])
dn.compile( loss='binary_crossentropy' ,optimizer=Adam( learning_rate=0.00001 ) ,metrics=[ 'accuracy' ])

In [12]:
forest1 = RandomForestClassifier(max_depth=7, min_samples_leaf=2, min_samples_split=2, n_estimators=95)
forest2 = RandomForestRegressor(max_depth=6, min_samples_leaf=2, min_samples_split=2, n_estimators=65)
gam1 = GAM1(40)
gam2 = GAM2(40)
nw = NadarayaWatsonCV(dict(kernel=["polynomial"],gamma=np.logspace(-1, 1, 100)),scoring='neg_mean_absolute_error')

In [ ]:
x_train, x_test, y_train, y_test = tts(X, y, test_size = 0.25, stratify=y)

In [ ]:
x_train2, x_test2, y_train2, y_test2 = tts(X, to_categorical(data['Churn'].values,2), test_size = 0.25, stratify=y)

In [13]:
def boosted(x, y, xnew, model1, model2, gam_first = False, gam_second = False, ynew = None):
  model1.fit(x,y)
  if gam_first:
    residuals1 = y - model1.predict(x,y)
  else:
    residuals1 = y - model1.predict(x)
  model2.fit(x,np.around(residuals1))
  if gam_first and gam_second:
    output = model1.predict(xnew,ynew) + model2.predict(xnew, ynew)
  elif gam_second:
    output = model1.predict(xnew) + model2.predict(xnew, ynew)
  elif gam_first:
    output = model1.predict(xnew, ynew) + model2.predict(xnew)
  else:
    output = model1.predict(xnew) + model2.predict(xnew)
  return output 

# Accuracies

Forest

In [ ]:
accuracy_score(y_test, np.around(boosted(x_train, y_train, x_test, gam1, forest1, True)))

0.9940047961630696

In [ ]:
accuracy_score(y_test, np.around(boosted(x_train, y_train, x_test, forest1, gam2)))

0.8920863309352518

In [ ]:
accuracy_score(y_test, np.around(boosted(x_train, y_train, x_test, nw, forest1)))

0.947242206235012

In [ ]:
accuracy_score(y_test, np.around(boosted(x_train, y_train, x_test, forest1, nw)))

0.9388489208633094

In [ ]:
accuracy_score(y_test2, np.around(boosted(x_train2, y_train2, x_test2, nn, forest2)))

27/27 [==============================] - 0s 2ms/step


0.935251798561151

In [ ]:
accuracy_score(y_test2, np.around(boosted(x_train2, y_train2, x_test2, dn, forest2)))

27/27 [==============================] - 0s 5ms/step


0.9376498800959233

GAM

In [ ]:
accuracy_score(y_test, np.around(boosted(x_train, y_train, x_test, gam1, nw, True)))

1.0

In [ ]:
accuracy_score(y_test, np.around(boosted(x_train, y_train, x_test, nw, gam2)))

0.9976019184652278

In [ ]:
accuracy_score(y_test2, np.around(boosted(x_train2, y_train2, x_test2, nn, gam2)))

27/27 [==============================] - 0s 2ms/step


ValueError: ignored

In [ ]:
accuracy_score(y_test2, np.around(boosted(x_train2, y_train2, x_test2, dn, gam2)))

27/27 [==============================] - 0s 6ms/step


ValueError: ignored

NW

In [ ]:
accuracy_score(y_test2, np.around(boosted(x_train2, y_train2, x_test2, nn, nw)))

27/27 [==============================] - 0s 3ms/step


0.854916067146283

In [ ]:
accuracy_score(y_test2, np.around(boosted(x_train2, y_train2, x_test2, dn, nw)))

27/27 [==============================] - 0s 5ms/step


0.854916067146283

In [15]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [16]:
polynomial_features= PolynomialFeatures(degree=3)
polynomial_features.fit(X)
x_poly = polynomial_features.transform(X)

## Feature Selection
### Lasso

In [17]:
sel_ = SelectFromModel(Lasso(alpha=0.01))
sel_.fit(scaler.fit_transform(x_poly), y)

SelectFromModel(estimator=Lasso(alpha=0.01))

In [18]:
x_poly_new = sel_.transform(x_poly)

In [19]:
x_poly_new.shape

(3333, 18)

In [20]:
features = sel_.get_feature_names_out()

In [21]:
features = [int(feature[1:]) for feature in features]

In [22]:
polynomial_features.get_feature_names_out()[features]

array(['x1', 'x5', 'x9', 'x1^2', 'x1 x5', 'x4^2', 'x0 x9^2', 'x1^3',
       'x1^2 x5', 'x1 x9^2', 'x2 x5^2', 'x3^3', 'x4^2 x6', 'x4 x5^2',
       'x5^3', 'x5^2 x8', 'x6 x8 x9', 'x9^3'], dtype=object)

# BEST: GAM + Forest, GAM + NW, NW + GAM

In [23]:
mses = []
acc = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)

for idxtrain, idxtest in kf.split(x_poly_new):
  xtrain = x_poly_new[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x_poly_new[idxtest]
  xtrain = scaler.fit_transform(xtrain)
  xtest = scaler.transform(xtest)
  
  boost = np.around(boosted(xtrain, ytrain, xtest, gam1, forest1, True, False, ytest))

  mses.append(mse(ytest,boost))
  acc.append(accuracy_score(ytest, np.around(boost)))

print("The new Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mses)))
print("Average accuracy:" + str(np.mean(acc)) + '%')
for i in acc:
  print(str(i) + '%')

The new Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 5916850789353784.0
Average accuracy:0.9912948877020733%
0.9970059880239521%
0.9970059880239521%
0.9940119760479041%
0.987987987987988%
0.996996996996997%
0.984984984984985%
0.990990990990991%
0.993993993993994%
0.990990990990991%
0.978978978978979%


In [24]:
mses = []
acc = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)

for idxtrain, idxtest in kf.split(x_poly_new):
  xtrain = x_poly_new[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x_poly_new[idxtest]
  xtrain = scaler.fit_transform(xtrain)
  xtest = scaler.transform(xtest)

  boost = np.around(boosted(xtrain, ytrain, xtest, gam1, nw, True, False, ytest))

  mses.append(mse(ytest,boost))
  acc.append(accuracy_score(ytest, np.around(boost)))

print("The new Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mses)))
print("Average accuracy:" + str(np.mean(acc)) + '%')
for i in acc:
  print(str(i) + '%')

The new Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 5916850789353784.0
Average accuracy:0.9912948877020733%
0.9970059880239521%
0.9970059880239521%
0.9940119760479041%
0.987987987987988%
0.996996996996997%
0.984984984984985%
0.990990990990991%
0.993993993993994%
0.990990990990991%
0.978978978978979%


In [25]:
mses = []
acc = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)

for idxtrain, idxtest in kf.split(x_poly_new):
  xtrain = x_poly_new[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = x_poly_new[idxtest]
  xtrain = scaler.fit_transform(xtrain)
  xtest = scaler.transform(xtest)

  boost = np.around(boosted(xtrain, ytrain, xtest, nw, gam2, False, True, ytest))

  mses.append(mse(ytest,boost))
  acc.append(accuracy_score(ytest, np.around(boost)))

print("The new Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : "+str(np.mean(mses)))
print("Average accuracy:" + str(np.mean(acc)) + '%')
for i in acc:
  print(str(i) + '%')

The new Cross-validated Mean Squared Error for Gramfort's Locally Weighted Regression is : 2297866720027721.0
Average accuracy:0.9657972343601087%
0.9640718562874252%
0.9670658682634731%
0.9640718562874252%
0.9579579579579579%
0.972972972972973%
0.96996996996997%
0.972972972972973%
0.9519519519519519%
0.987987987987988%
0.948948948948949%
